In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, from_json
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
import sys

# Create a Spark session with Kafka integration
spark = SparkSession.builder \
    .appName("KafkaStreamExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-48de0c52-0995-434e-931a-83a329c56745;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 420ms :: artifacts dl 22ms
	

In [2]:
# Define schema for the JSON data (expecting a key 'message' with a string value)
json_schema = StructType([
    StructField("message_content", StringType(), True)  # The 'message' key contains a sentence
])

In [3]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9093") \
    .option("subscribe", "test-topic") \
    .load()

In [4]:
# The value in Kafka is in bytes, so we need to cast it to string
kafka_stream_df = df.selectExpr("CAST(value AS STRING)")

In [5]:
# Parse the JSON data and extract the 'message' field
parsed_df = kafka_stream_df.select(from_json(col("value"), json_schema).alias("data"))

In [6]:
# Extract the 'message' field from the parsed JSON
message_df = parsed_df.select(col("data.message_content").alias("message"))

In [7]:
# Split the message into words
words_df = message_df.select(explode(split(col("message"), "\s+")).alias("word"))

In [8]:
# Count the occurrences of each word
word_counts_df = words_df.groupBy("word").count()

In [9]:
# Write the output to the console, this will update in real-time
query = word_counts_df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

25/12/11 20:34:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0943463c-4e0c-4f7f-aa92-bd0182a5e9a1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/12/11 20:34:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/12/11 20:34:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----+
|word    |count|
+--------+-----+
|colorful|1    |
|slowly. |1    |
|dog     |1    |
|flies   |1    |
|The     |1    |
+--------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |1    |
|colorful   |1    |
|slowly.    |2    |
|jumps      |2    |
|dog        |1    |
|gracefully.|1    |
|eats       |1    |
|ball       |2    |
|flies      |1    |
|small      |1    |
|The        |4    |
|loudly.    |1    |
|car        |1    |
|big        |1    |
+-----------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |2    |
|quickly.   |1    |
|colorful   |1    |
|slowly.    |3    |
|jumps      |3    |
|dog        |1    |
|gracefully.|2    |
|eats       |1    |
|fast       |1    |
|ball       |3    |
|runs       |1    |
|flies      |1    |
|sings      |1    |
|bird       |1    |
|small      |2    |
|The        |7    |
|loudly.    |1    |
|car        |2    |
|big        |1    |
+-----------+-----+



...

-------------------------------------------
Batch: 240
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |104  |
|quickly.   |79   |
|colorful   |88   |
|slowly.    |104  |
|jumps      |117  |
|dog        |104  |
|gracefully.|81   |
|eats       |82   |
|cat        |86   |
|fast       |93   |
|ball       |87   |
|runs       |91   |
|flies      |88   |
|sings      |81   |
|hungrily.  |85   |
|bird       |87   |
|small      |79   |
|The        |459  |
|loudly.    |110  |
|car        |95   |
+-----------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 241
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |105  |
|quickly.   |80   |
|colorful   |89   |
|slowly.    |105  |
|jumps      |118  |
|dog        |104  |
|gracefully.|81   |
|eats       |83   |
|cat        |87   |
|fast       |93   |
|ball       |87   |
|runs       |91   |
|flies      |88   |
|sings      |81   |
|hungrily.  |85   |
|bird       |87   |
|small      |79   |
|The        |461  |
|loudly.    |110  |
|car        |96   |
+-----------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 242
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |106  |
|quickly.   |81   |
|colorful   |89   |
|slowly.    |106  |
|jumps      |119  |
|dog        |104  |
|gracefully.|81   |
|eats       |83   |
|cat        |87   |
|fast       |93   |
|ball       |89   |
|runs       |91   |
|flies      |89   |
|sings      |81   |
|hungrily.  |85   |
|bird       |87   |
|small      |80   |
|The        |463  |
|loudly.    |110  |
|car        |96   |
+-----------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 243
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |106  |
|quickly.   |81   |
|colorful   |90   |
|slowly.    |107  |
|jumps      |119  |
|dog        |105  |
|gracefully.|81   |
|eats       |85   |
|cat        |87   |
|fast       |94   |
|ball       |89   |
|runs       |91   |
|flies      |89   |
|sings      |81   |
|hungrily.  |86   |
|bird       |88   |
|small      |80   |
|The        |465  |
|loudly.    |110  |
|car        |96   |
+-----------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 244
-------------------------------------------
+-----------+-----+
|word       |count|
+-----------+-----+
|lazy       |106  |
|quickly.   |81   |
|colorful   |91   |
|slowly.    |108  |
|jumps      |119  |
|dog        |106  |
|gracefully.|81   |
|eats       |85   |
|cat        |87   |
|fast       |94   |
|ball       |89   |
|runs       |91   |
|flies      |89   |
|sings      |82   |
|hungrily.  |86   |
|bird       |88   |
|small      |80   |
|The        |466  |
|loudly.    |110  |
|car        |96   |
+-----------+-----+
only showing top 20 rows



In [ ]:
spark.stop()